In [148]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [149]:
import logging
import delta_sharing
import pandas as pd

import general_functions.databricks_client as db_client
from general_functions.call_api_with_account_id import send_to_innkeepr_api_paginated
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url

In [150]:
customer = "Nikin"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]

In [151]:
# Load example data for cate
cate = pd.read_csv("SprintStories/EN_2730_Nikin/cate_20250905_test.csv", index_col=0)
cate

In [152]:
cate_with_nan_profiles = cate[cate["profile_id"].isnull()]
cate_with_nan_profiles

# Check if cate.anonymousIds are in the profiles collection and profile.ids

In [201]:
#check if cate anonymousIds with null profile_id is in database
sample = 10000
anonymousId_list = cate_with_nan_profiles["anonymousId"].dropna().unique().tolist()
print(f"length of anonymousId_list = {len(anonymousId_list)}")
anonymousId_list_sample = anonymousId_list[:sample]
print(f"length of anonymousId_list_sample = {len(anonymousId_list_sample)}")

In [202]:
profiles = send_to_innkeepr_api_paginated(
    f"{url}/profiles/query",
    account_id,
    {"anonymousId": anonymousId_list_sample},
    logging
)
profiles = pd.json_normalize(profiles)
profiles.head()

In [203]:
profiles_pp = profiles.copy()
cookieid = "id"
col_name = "name"
data_col_extIds = "externalIds"
profiles_id="_id"
col_last_modified="lastModified"
col_user_id="anonymousId"
profiles_pp = profiles.rename(columns={"id": profiles_id})
profiles_pp = profiles_pp[[profiles_id, col_user_id, data_col_extIds, col_last_modified]]
profiles_pp = profiles_pp.explode(data_col_extIds)
profiles_pp[[cookieid, col_name]] = profiles_pp[data_col_extIds].apply(
        lambda x: pd.Series([x[cookieid], x[col_name]])
    )
profiles_pp

In [155]:
vc_profile_id_is_null = profiles["id"].isnull().sum()
if vc_profile_id_is_null == 0:
    print(f"No null values")
    profiles_null = pd.DataFrame()
else:
    profiles_null = profiles[profiles["id"].isnull()]
    print(profiles_null)

# Check profile views

In [163]:
profile_path = db_client.return_databricks_client()
table_path = f"{profile_path}#delta_share_events.{account_id}.profiles_view_30_outlook" #features_view_30_outlook"
profiles_view = delta_sharing.load_as_pandas(table_path) #, limit=20000)

In [164]:
profiles_view.to_csv("SprintStories/EN_2730_Nikin/profiles_view_30_outlook.csv", index=False)

In [165]:
#check for missing profile id
profiles_view[profiles_view["anonymousId"]=="689252e2a7f5487f824ba2ed"]

In [ ]:
profiles_in_cate = profiles_view[profiles_view["anonymousId"].isin(anonymousId_list)]
profiles_in_cate.head()

In [ ]:
unique_anonymous_ids_profiles_cate = profiles_in_cate["anonymousId"].unique().tolist()
print(f"length of unique_anonymous_ids_profiles_cate = {len(unique_anonymous_ids_profiles_cate)}")
print(f"length of anonymousId_list = {len(anonymousId_list)}")


In [ ]:
profiles_in_cate_profile_id_is_null = profiles_in_cate[profiles_in_cate["profile_id"].isnull()]
if len(profiles_in_cate_profile_id_is_null) == 0:
    print(f"Profiles in Databricks Profile View are fine")
else:
    print(f"Error in Databricks Profile View")
    print(profiles_in_cate_profile_id_is_null)

# Check data after loading conversion table

In [ ]:
conv_table = pd.read_csv("SprintStories/EN_2730_Nikin/targeted_user_conv_table.csv", index_col=0)
conv_table.head()

In [ ]:
conv_table_with_null_anonymousId = conv_table[conv_table["anonymousId"].isnull()]
if len(conv_table_with_null_anonymousId) == 0:
    print(f"No null values")
else:
    print(f"Error in Targeted User Conversion Table")
    print(conv_table_with_null_anonymousId)

# Check data after loading conversion table & match with profiles

In [ ]:
conv_table_with_profiles = pd.read_csv("SprintStories/EN_2730_Nikin/targeted_user_profile_match.csv", index_col=0)
conv_table_with_profiles.head()

In [ ]:
unique_anonymousIds_conv_table_with_profiles = conv_table_with_profiles["anonymousId"].nunique()
unique_anonymousIds_conv_table_with_profiles

In [ ]:
conv_table_anonmyousId_are_in_profile_view = conv_table_with_profiles[conv_table_with_profiles["anonymousId"].isin(profiles_view["anonymousId"])]
conv_table_anonmyousId_are_in_profile_view_unique = conv_table_anonmyousId_are_in_profile_view["anonymousId"].nunique()
if conv_table_anonmyousId_are_in_profile_view_unique != unique_anonymousIds_conv_table_with_profiles:
    print(f"Some error: Missing profiles in Databricks Profile View expected: {unique_anonymousIds_conv_table_with_profiles}, actual: {conv_table_anonmyousId_are_in_profile_view_unique}")

In [ ]:
missing_profiles = conv_table_with_profiles[~conv_table_with_profiles["anonymousId"].isin(profiles_view["anonymousId"])]
missing_profiles

In [ ]:
missing_profiles["created"].value_counts()

In [ ]:
profiles_view[profiles_view["anonymousId"]=="6891a6706a367afb6217ae27"]

# Simulate profiles query

In [204]:
simulation_date = "2025-09-04"
max_date = "2025-09-05"

In [205]:
try:
    db_views_30=pd.read_csv("SprintStories/EN_2730_Nikin/db_views_30.csv", index_col=0)
except:
    print("load from databricks")
    profile_path = db_client.return_databricks_client()
    table_path = f"{profile_path}#delta_share_events.{account_id}.features_view_30_outlook" #features_view_30_outlook"
    db_views_30 = delta_sharing.load_as_pandas(table_path) #, limit=20000)
    db_views_30.to_csv("SprintStories/EN_2730_Nikin/db_views_30.csv", index=False)

In [206]:
db_views_30=db_views_30.reset_index()

In [223]:
db_views_30 = db_views_30[["created","anonymousId","session"]]
db_views_filtered = db_views_30[db_views_30["created"] >= simulation_date]
db_views_filtered = db_views_filtered[db_views_filtered["created"] < max_date]
print(f"Date range filtered: {db_views_filtered['created'].min()} to {db_views_filtered['created'].max()}")
print(f"Number of unique anonymousIds: {db_views_filtered['anonymousId'].nunique()}")

In [208]:
db_views_30_not_in_profile_views = db_views_filtered[~db_views_filtered["anonymousId"].isin(profiles_view["anonymousId"].unique().tolist())]
db_views_30_not_in_profile_views

In [209]:
missing_profiles = db_views_30_not_in_profile_views["anonymousId"].unique()
print(f"length of missing_profiles = {len(missing_profiles)}")

In [210]:
print(f"Query for date greater than: {simulation_date}")
profiles_db = send_to_innkeepr_api_paginated(
    f"{url}/profiles/query",
    account_id,
    {"lastModified":{"$gte":"2025-09-04"}},
    logging
)
len(profiles_db)

In [211]:
profiles_db = pd.json_normalize(profiles_db)
profiles_db.head()

In [212]:
profiles_db["lastModified"].min(), profiles_db["lastModified"].max()

In [213]:
profiles_db["anonymousId"].nunique()

In [214]:
found_profiles = db_views_filtered[db_views_filtered["anonymousId"].isin(profiles_db["anonymousId"].unique().tolist())]
found_profiles["anonymousId"].nunique()

In [215]:
missing_profiles_in_response = db_views_filtered[~db_views_filtered["anonymousId"].isin(profiles_db["anonymousId"].unique().tolist())]
missing_profiles_in_response

In [216]:
missing_profiles_pd = profiles_db[profiles_db["anonymousId"].isin(missing_profiles).fillna(False)]
missing_profiles_pd

## test transformations

In [217]:
cookieid = "id"
col_name = "name"
data_col_extIds = "externalIds"
profiles_id="_id"
col_last_modified="lastModified"
col_user_id="anonymousId"

In [225]:
list_unique_anonymousids = db_views_filtered["anonymousId"].unique().tolist()
len(list_unique_anonymousids)

In [218]:
profiles = profiles_db.copy()
profiles = profiles.rename(columns={"id": profiles_id})
profiles = profiles[[profiles_id, col_user_id, data_col_extIds, col_last_modified]]
profiles = profiles.explode(data_col_extIds)
profiles[[cookieid, col_name]] = profiles[data_col_extIds].apply(
        lambda x: pd.Series([x[cookieid], x[col_name]])
    )
profiles

In [222]:
profiles["anonymousId"].nunique()

In [227]:
profiles = profiles[profiles[col_user_id].isin(list_unique_anonymousids)]

In [228]:
profiles["anonymousId"].nunique()

In [230]:
test = profiles[
        (profiles[col_name].str.contains("email_sha256"))
        | (profiles[col_name].str.contains("email"))
        | (profiles[col_name].str.contains("_ga"))
        | (profiles[col_name].str.contains("_fbp"))
        | (profiles[col_name].str.contains("criteo_gum_id"))
    ]

In [234]:
test["anonymousId"].nunique()

In [233]:
print(test[test["anonymousId"].isin(missing_profiles)]["anonymousId"].nunique())
test[test["anonymousId"].isin(missing_profiles)]["name"].value_counts()

In [199]:
profiles[profiles["anonymousId"]=="68b8ffee5287866a2c7a70fc"]

In [200]:
profiles_db[profiles_db["anonymousId"]=="68b8ffee5287866a2c7a70fc"]